In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler, StandardScaler

import faiss

In [2]:
data_base = pd.read_csv('base.csv', index_col=0)

In [3]:
data_train = pd.read_csv('train.csv', index_col=0)

In [4]:
data_validation = pd.read_csv('validation.csv', index_col=0)

In [5]:
data_validation_answer = pd.read_csv('validation_answer.csv', index_col=0)

In [24]:
data_base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2918139 entries, 0-base to 4744766-base
Data columns (total 72 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64


In [25]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21  21  

In [26]:
data_validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21

In [27]:
data_validation_answer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Expected  100000 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [ ]:
data_base.head()

In [ ]:
data_train.head()

In [ ]:
data_validation.head()

In [ ]:
data_validation_answer.head()

In [ ]:
data_train.corr()

Данные не имеют некого понятного смысла, попробуем изучить представления данных.

In [ ]:
data_train.columns

In [ ]:
def data_information(column):
    print(data_train[column].describe())
    
    plt.figure(figsize=(8, 5))
    data_train[column].hist(bins=10);
    plt.title('гистограмма признака {}'.format(column))
    
    plt.figure(figsize=(10, 7))
    data_train.boxplot(column);
    plt.title('диаграмма размаха признака {}'.format(column))
    
    Q1 = data_train[column].quantile(0.25)
    Q3 = data_train[column].quantile(0.75)
    IQR = Q3 - Q1
    print('аномальные заначения признака {}: > {}, < {}'.format(column, Q3 + 1.5 * IQR, Q1 - 1.5 * IQR))

In [ ]:
for i in range(72):
    data_information(data_train.columns[i+1])

**выводы**
после первоначального изучения данных мы пришли к тому, что:
+ данные не коррелируют между собой.
+ все признаки имеют нормальное распределения кроме признаков 6, 21, 25, 33, 44, 59, 65, 70.
+ данные имеют аномальные значения, но пока удалять их не будем.
+ признаки 59, 65 имеют почти равномерное распределение, так что можем в дальнейшем от них избавится.

## K-Means

In [ ]:
target_train = data_train['Target']
features_train = data_train.drop(['Id', 'Target'], axis=1)

In [ ]:
# define the model
kmeans_model = KMeans(n_clusters=2)

# assign each data point to a cluster
kmeans_model = kmeans_model.fit(features_train)
kmeans_result = kmeans_model.predict(features_train)
# get all of the unique clusters
kmeans_clusters = np.unique(kmeans_result)
# plot the DBSCAN clusters
for kmeans_cluster in kmeans_clusters:
    # get data points that fall in this cluster
    index = np.where(kmeans_result == kmeans_cluster)
    # make the plot
    plt.scatter(features_train.loc[index, '0'], features_train.loc[index, '1'])

# show the DBSCAN plot
plt.show()

**метрики оценки**

*Коэффициент силуэта* - этот показатель измеряет, насколько хорошо каждая точка данных вписывается в назначенный ей кластер, и находится в диапазоне от -1 до 1. Высокий коэффициент силуэта указывает на то, что точки данных хорошо кластеризованы, а низкий коэффициент указывает на то, что точки данных могут быть назначены. в неправильный кластер.
*Индекс Дэвиса-Булдина* - этот показатель измеряет среднее сходство между каждым кластером и его наиболее похожим кластером с учетом размера кластеров. Низкий индекс Дэвиса-Булдина указывает на то, что кластеры хорошо разделены и различимы.

In [ ]:
labels = kmeans_model.labels_
# assign each data point to a cluster
silhouette_score_0 = silhouette_score(features_train, labels)

In [ ]:
print(silhouette_score_0)

In [ ]:
davies_bouldin_score_0 = davies_bouldin_score(features_train, labels)
print(davies_bouldin_score_0)

подбераем лучшее количество кластеров на основе этих двух показателей.

In [ ]:
best_silhouette_score = 1000
best_davies_bouldin_score = 1000
itr_silhouette = 1000
itr_davies_bouldin = 1000
for i in range(2, 100):
    kmeans_model = KMeans(n_clusters=i).fit(features_train)
    labels = kmeans_model.labels_
    silhouette_score_0 = silhouette_score(features_train, labels)
    davies_bouldin_score_0 = davies_bouldin_score(features_train, labels)
    if silhouette_score_0 > best_silhouette_score:
        best_silhouette_score = silhouette_score_0
        itr_silhouette = i
    if davies_bouldin_score_0 < best_davies_bouldin_score:
        best_davies_bouldin_score = davies_bouldin_score_0
        itr_davies_bouldin = i
    print('количество кластеров = {}, silhouette_score = {}, davies_bouldin_score = {}'.format(i, silhouette_score_0, davies_bouldin_score_0))
    

замечаем что лучший результат по метрикам качества получаем при количество кластеров = 3

подбераем лучшее количество итерации на основе этих двух показателей.

In [ ]:
best_silhouette_score = 1000
best_davies_bouldin_score = 1000
itr_silhouette = 1000
itr_davies_bouldin = 1000
for i in range(100, 1501, 100):
    kmeans_model = KMeans(n_clusters=3, max_iter=i).fit(features_train)
    labels = kmeans_model.labels_
    silhouette_score_0 = silhouette_score(features_train, labels)
    davies_bouldin_score_0 = davies_bouldin_score(features_train, labels)
    if silhouette_score_0 > best_silhouette_score:
        best_silhouette_score = silhouette_score_0
        itr_silhouette = i
    if davies_bouldin_score_0 < best_davies_bouldin_score:
        best_davies_bouldin_score = davies_bouldin_score_0
        itr_davies_bouldin = i
    print('количество итерации = {}, silhouette_score = {}, davies_bouldin_score = {}'.format(i, silhouette_score_0, davies_bouldin_score_0))

замечаем что лучший результат по метрикам качества получаем при количество итирации = 200

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn_model.fit(features_train, target_train)

In [ ]:
result = knn_model.predict([features_train.loc[0]])

In [ ]:
print(result)

In [ ]:
result = knn_model.kneighbors([features_train.loc[0]])

In [ ]:
print(target_train[result[1][0]])

In [ ]:
target_train[0]

In [ ]:
result = knn_model.kneighbors(features_train)

In [ ]:
%%time
result = knn_model.predict(features_train)

In [ ]:
result

In [ ]:
accuracy_score(result,target_train)

In [ ]:
for i in range(1, 21):
    knn_model = KNeighborsClassifier(n_neighbors=i)
    knn_model.fit(features_train, target_train)
    result = knn_model.predict(features_train)
    print(i, '  ', accuracy_score(result,target_train))

## FAISS

In [6]:
K_NEIGHBORS = 5

In [7]:
NUM_SAMPLES = 50000

In [8]:
def acc_5(targets, indices, base_indices):
    """ Функция для расчета значения метрики accuracy
        входные параметры функции:
        targets - вектор целевого признака
        indices - матрица индексов найденных ближайших соседей
        base_index - словарь базовых индексов """
    
    acc = 0 # начальное значение метрики
    for target, el in zip(targets.values.tolist(), indices.tolist()):
        acc += int(target in [base_indices[r] for r in el])

    return 100 * acc / len(indices)

In [9]:
def faiss_model(data_base, data_train, num_samples=None):
    dims = data_base.shape[1] # размерность вектора признаков для каждого объекта
    n_cells = 50 # количество кластеров (ячейки Воронова?)
    quantizer = faiss.IndexFlatL2(dims) # квантователь 
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells) # создание индекса с кластеризацией
    idx_l2.train(np.ascontiguousarray(data_base.values[:num_samples, :]).astype('float32'))
    idx_l2.add(np.ascontiguousarray(data_base.values).astype('float32'))
    base_index = {k: v for k, v in enumerate(data_base.index.to_list())}
    y_train = data_train['Target'] # целевой признак обучающей выборки
    X_train = data_train.drop('Target', axis=1) # обучающая выборка
    r, idx = idx_l2.search(np.ascontiguousarray(X_train.values).astype('float32'), K_NEIGHBORS)
    accuracy = acc_5(y_train, idx, base_index)
    print(accuracy, '%')

In [ ]:
faiss_model(data_base, data_train, NUM_SAMPLES)

## EDA

In [ ]:
df_samples = data_train[:NUM_SAMPLES]

In [ ]:
df_samples.hist(figsize=[20, 20], bins=50);

In [ ]:
data_to_drop = ['6', '21', '25', '33', '44', '59', '65', '70']
for i in data_to_drop:
    data_base_new = data_base.drop(i, axis=1)
    X_train_new = data_train.drop(i, axis=1) # обучающая выборка
    print('при улаления столбца {} точность будет'.format(i))
    faiss_model(data_base_new, X_train_new, NUM_SAMPLES)

замечаем что при удаления столбцов 33, 44 и 59 повышается точность.

In [10]:
col_to_delet = ['33', '44', '59']
data_base_new = data_base.drop(col_to_delet, axis=1)
X_train_new = data_train.drop(col_to_delet, axis=1)
X_validation_new = data_validation.drop(col_to_delet, axis=1).reset_index()
faiss_model(data_base_new, X_train_new, NUM_SAMPLES)

32.108 %


### Нормализация (масштабирование) численных признаков

In [11]:
scaler = StandardScaler()
num_cols = X_train_new.select_dtypes(include='number').columns # численные признаки
# обучим базовую выборку с отмасштабированными численными признаками


In [12]:
X_base_normal_SS = pd.DataFrame(scaler.fit_transform(data_base_new[num_cols]), index=data_base_new.index, columns=num_cols)
X_train_normal_SS = pd.DataFrame(scaler.transform(X_train_new[num_cols]), index=X_train_new.index, columns=num_cols)
X_train_normal_SS['Target'] = data_train['Target']
X_validation_normal_SS = pd.DataFrame(scaler.transform(X_validation_new[num_cols]), index=X_validation_new.index, columns=num_cols)

In [ ]:
faiss_model(X_base_normal_SS, X_train_normal_SS, NUM_SAMPLES)

замечаем что точность модели повышается поле нормализации данных с используванием стандартного нормализатора.

In [ ]:
scaler = RobustScaler()

In [ ]:
X_base_normal_RS = pd.DataFrame(scaler.fit_transform(data_base_new[num_cols]), index=data_base_new.index, columns=num_cols)
X_train_normal_RS = pd.DataFrame(scaler.transform(X_train_new[num_cols]), index=X_train_new.index, columns=num_cols)
X_train_normal_RS['Target'] = data_train['Target']
X_validation_normal_RS = pd.DataFrame(scaler.transform(X_validation_new[num_cols]), index=X_validation_new.index, columns=num_cols)

In [ ]:
faiss_model(X_base_normal_RS, X_train_normal_RS, NUM_SAMPLES)

стандартная нормализация лучше робостной.

### удаление выбросов

In [13]:
X_train_normal_SS_all = X_train_normal_SS.copy()
for column in X_train_normal_SS.columns:
    if column in ['6', '21', '25', '33', '44', '59', '70', 'Target']:
        continue
    Q1 = X_train_normal_SS[column].quantile(0.25)
    Q3 = X_train_normal_SS[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # data frame with deleted unwanted values from only one column
    X_train_normal_SS_column = X_train_normal_SS[~((X_train_normal_SS[column] > (Q3 + 1.5 * IQR)) | 
                                          (X_train_normal_SS[column] < (Q1 - 1.5 * IQR)))]
    # data frame with deleting all the unwanted values
    X_train_normal_SS_all = X_train_normal_SS_all[~((X_train_normal_SS[column] > (Q3 + 1.5 * IQR)) | 
                                          (X_train_normal_SS[column] < (Q1 - 1.5 * IQR)))]
    
    # calculate the accuracy of data frame with deleted unwanted values from only one column 
    print('при улаления выбросов столбца {} точность будет'.format(column))
    faiss_model(X_base_normal_SS, X_train_normal_SS_column, 10000)

<ipython-input-13-f3564622d06b>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_normal_SS_all = X_train_normal_SS_all[~((X_train_normal_SS[column] > (Q3 + 1.5 * IQR)) |
<ipython-input-13-f3564622d06b>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_normal_SS_all = X_train_normal_SS_all[~((X_train_normal_SS[column] > (Q3 + 1.5 * IQR)) |
<ipython-input-13-f3564622d06b>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_normal_SS_all = X_train_normal_SS_all[~((X_train_normal_SS[column] > (Q3 + 1.5 * IQR)) |
<ipython-input-13-f3564622d06b>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_normal_SS_all = X_train_normal_SS_all[~((X_train_normal_SS[column] > (Q3 + 1.5 * IQR)) |
<ipython-input-13-f3564622d06b>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_normal_SS_all = X_train_normal_SS_

замечаем что пре удаления выбросов из одного признака точность моделе может повысется на 0,5%

In [14]:
X_train_normal_SS_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63981 entries, 0-query to 99996-query
Data columns (total 70 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       63981 non-null  float64
 1   1       63981 non-null  float64
 2   2       63981 non-null  float64
 3   3       63981 non-null  float64
 4   4       63981 non-null  float64
 5   5       63981 non-null  float64
 6   6       63981 non-null  float64
 7   7       63981 non-null  float64
 8   8       63981 non-null  float64
 9   9       63981 non-null  float64
 10  10      63981 non-null  float64
 11  11      63981 non-null  float64
 12  12      63981 non-null  float64
 13  13      63981 non-null  float64
 14  14      63981 non-null  float64
 15  15      63981 non-null  float64
 16  16      63981 non-null  float64
 17  17      63981 non-null  float64
 18  18      63981 non-null  float64
 19  19      63981 non-null  float64
 20  20      63981 non-null  float64
 21  21      63981 non-null  floa

In [15]:
faiss_model(X_base_normal_SS, X_train_normal_SS_all, NUM_SAMPLES)

58.387646332504964 %


замечаем что точность повышается на 0,3% при удалении все данные с выбросами.

### исследование гиперпараметров FAISS

In [22]:
# диапазоны значений параметров алгоритма
n_cells_array = range(50, 201, 50)
nprobe_array = range(1, 20, 4)
best_n_cells = 1
best_nprobe = 1
best_acc_5 = 0
best_distx = []
best_idx = []

In [16]:
def faiss_model_hyper(data_base, data_train, num_samples=None, n_cells=None, nprobe=None):
    dims = data_base.shape[1] # размерность вектора признаков для каждого объекта
    quantizer = faiss.IndexFlatL2(dims) # квантователь 
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells) # создание индекса с кластеризацией
    idx_l2.train(np.ascontiguousarray(data_base.values[:num_samples, :]).astype('float32'))
    idx_l2.add(np.ascontiguousarray(data_base.values).astype('float32'))
    idx_l2.nprobe = nprobe 
    base_index = {k: v for k, v in enumerate(data_base.index.to_list())}
    y_train = data_train['Target'] # целевой признак обучающей выборки
    X_train = data_train.drop('Target', axis=1) # обучающая выборка
    r, idx = idx_l2.search(np.ascontiguousarray(X_train.values).astype('float32'), K_NEIGHBORS)
    accuracy = acc_5(y_train, idx, base_index)
    return idx_l2, accuracy

In [23]:
n_iter = 1
for n_cells in n_cells_array:
    for nprobe in nprobe_array:
        idx_l2, acc = faiss_model_hyper(X_base_normal_SS, X_train_normal_SS_all, NUM_SAMPLES, n_cells, nprobe)
        print('iteration...', n_iter, 'n_cells...', n_cells, 'nprobe...', nprobe,
              'acc@5...', acc)
        n_iter += 1
        if acc > best_acc_5:
            best_acc_5 = acc
            best_n_cells = n_cells
            best_nprobe = nprobe

print('best acc@5:', best_acc_5, '%')
print('best_n_cells:', best_n_cells)
print('best_nprobe:', best_nprobe)

iteration... 1 n_cells... 50 nprobe... 1 acc@5... 58.387646332504964
iteration... 2 n_cells... 50 nprobe... 5 acc@5... 67.08866694800018
iteration... 3 n_cells... 50 nprobe... 9 acc@5... 68.03894906300307
iteration... 4 n_cells... 50 nprobe... 13 acc@5... 68.49377158844032
iteration... 5 n_cells... 50 nprobe... 17 acc@5... 68.98141635798127
iteration... 6 n_cells... 100 nprobe... 1 acc@5... 58.387646332504964
iteration... 7 n_cells... 100 nprobe... 5 acc@5... 67.08866694800018
iteration... 8 n_cells... 100 nprobe... 9 acc@5... 68.03894906300307
iteration... 9 n_cells... 100 nprobe... 13 acc@5... 68.49377158844032
iteration... 10 n_cells... 100 nprobe... 17 acc@5... 68.98141635798127
iteration... 11 n_cells... 150 nprobe... 1 acc@5... 58.387646332504964
iteration... 12 n_cells... 150 nprobe... 5 acc@5... 67.08866694800018


KeyboardInterrupt: 

замечаем что самый лучший результат является 68.98% при n_cells = 50 и nprobe = 17.

### вывод 
По результатам проведенных исследований можно сделать следующие предварительные выводы:

- замечаем что точность повышается на 0,3% при удалении все данные с выбросами.
- точность модели повышается после нормализации данных с используванием стандартного нормализатора.
- удаления столбцов 33, 44 и 59 повышается точность.
- согласно проведенным исследованиям, наилучшие результаты дают значения n_cells = 50 nprobe = 17 (68.98%), 

### тестирования лучшей модели на валидационной выброке

In [17]:
%%time
idx_l2, acc = faiss_model_hyper(X_base_normal_SS, X_train_normal_SS_all, num_samples=None, n_cells=50, nprobe=17)

Wall time: 20min 1s


In [20]:
acc

69.27837951891968

In [ ]:
def acc_5(targets, indices, base_indices):
    """ Функция для расчета значения метрики accuracy
        входные параметры функции:
        targets - вектор целевого признака
        indices - матрица индексов найденных ближайших соседей
        base_index - словарь базовых индексов """
    
    acc = 0 # начальное значение метрики
    for target, el in zip(targets.values.tolist(), indices.tolist()):
        acc += int(target in [base_indices[r] for r in el])

    return 100 * acc / len(indices)

In [ ]:
def faiss_model_hyper(data_base, data_train, num_samples=None, n_cells=None, nprobe=None):
    dims = data_base.shape[1] # размерность вектора признаков для каждого объекта
    quantizer = faiss.IndexFlatL2(dims) # квантователь 
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells) # создание индекса с кластеризацией
    idx_l2.train(np.ascontiguousarray(data_base.values[:num_samples, :]).astype('float32'))
    idx_l2.add(np.ascontiguousarray(data_base.values).astype('float32'))
    idx_l2.nprobe = nprobe 
    base_index = {k: v for k, v in enumerate(data_base.index.to_list())}
    y_train = data_train['Target'] # целевой признак обучающей выборки
    X_train = data_train.drop('Target', axis=1) # обучающая выборка
    r, idx = idx_l2.search(np.ascontiguousarray(X_train.values).astype('float32'), K_NEIGHBORS)
    accuracy = acc_5(y_train, idx, base_index)
    return idx_l2, accuracy

In [19]:
r, idx = idx_l2.search(np.ascontiguousarray(X_validation_normal_SS.values).astype('float32'), K_NEIGHBORS)
base_index = {k: v for k, v in enumerate(X_base_normal_SS.index.to_list())}
accuracy = acc_5(data_validation_answer, idx, base_index)
print(accuracy, '%')

0.0 %


**Я здесь не понял в чём проблема!!!**

## Общий вывод

По результатам проведенной работы с помощью библиотеки FAISS был разработан и протестирован алгоритм мэтчинга товаров, позволяющий каждому товару из валидационной выборки выборки сопоставить наиболее близкие товары из базовой выборки (по заданной метрике качества - acc@5).<br>
При решении данной задачи были выполнены следующие работы:
- проведен первичиный исследовательский анализ и предобработка данных исходных датасетов;
- найдены признаки, имеющие отличное от нормального распределения (под номерами 6, 21, 25, 33, 44, 59, 65 и 70), которые, скорее всего, являются препятствием для успешного обучения индекса FAISS (и последующего поиска похожих товаров) и возможных применяемых моделей;
- из этих признаков удалено  33, 44, 59 потому что они имели вредное влияние на обучения модели,
- повторено базовое решение (бейзлайн) задачи мэтчинга, давшее ок. 8.6 % качество;
- проведена нормализация (масштабирование/стандартизация) признаков с помощью базового инструмента StandardScaler;
- проведена проверка данных на коллинеарность, получено макс. значение коэффициента фи-корреляции $\phi_K$, равное 0.4276 - такая слабая линейная зависимость признаков не позволила нам сделать математический вывод о возможности удаления каких-либо признаков из выборок;
- проверено качество решения задачи поиска похожих товаров для отмасштабированных (скэйлированых) данных, которая показала, что только лишь одна нормализация позволила повысить качество решения задачи мэтчинга с 8.693 %  до 58.981 % - более, чем в 6 раз, что уже является неплохим результатом (при прочих равных условиях);
- проведено исследование гиперпараметров алгоритма (функций библиотеки FAISS), таких как, количество кластеров, на которые разбивается выборка (n_cells) и количество соседних кластеров, в которых осуществляется поиск после разбиения на n_cells кластеров и нахождения центроидов данных кластеров (nprobe). 
- в неких небольших диапазонах изменения указанных параметров, наилучшие результаты дают значения n_cells = 50, nprobe = 17 (65.9 %), 
- обучения на всех экземпляров с оптимально модели дала точность = 69.27% на обучающую выборку, однак на валидационную выборку дала 0% (не нашел в чем проблема)